In [1]:
import json
import pandas as pd

In [98]:
def cargar_json(path):
    try:
        with open(path, 'r', encoding='UTF-8') as f:
            return json.load(f)
    except:
        return None

def select_products(data):
    if data:
        products = list(dict(p) for p in data.get('data').get('data')[0].get('products'))
        return products


def transform_data(p, _tienda):
    # nombre
    nombre = p.get("summary") or p.get('productName')

    # precio (priorizo finalPrice; si no, webPrice)
    precio_raw = p.get("finalPrice") or p.get("webPrice")
    try:
        precio = float(precio_raw) if precio_raw is not None else None
    except (TypeError, ValueError):
        precio = None

    moneda = p.get("currencyCode")  # e.g., "ARS"
    simbolo = p.get("currencySymbol")  # e.g., "$"

    # estrellas y cantidad de reviews
    try:
        estrellas = float(p.get("ratingStar")) if p.get("ratingStar") is not None else None
    except (TypeError, ValueError):
        estrellas = None

    reviews = p.get("commentCount")

    # descripcion
    descripcion = (
        p.get("marketingLongDescription")
        or p.get("marketingShortDescription")
        or p.get("description")
    )

    # especificaciones: de la lista "classification" armo dict {a: b}
    especs = {}
    for item in p.get("classification", []) or []:
        k = item.get("a")
        v = item.get("b")
        if k:
            especs[k] = v

    # categoría: uso un nombre legible si existe; dejo también el path crudo
    categoria = p.get("flName") or p.get("flCode")  # "Computadoras de Escritorio"
    categoria_path = p.get("categoryPath")  # lista jerárquica si querés

    # imagen producto
    imagen = None
    media = p.get("media", {})
    if "thumbnail" in media and media["thumbnail"].get("imageAddress"):
        imagen = media["thumbnail"]["imageAddress"]
    elif "gallery" in media and len(media["gallery"]) > 0:
        imagen = media["gallery"][0].get("imageAddress")

    return {
        "nombre": nombre,
        'tienda': _tienda,
        "precio": precio,
        "moneda": moneda,
        "simbolo": simbolo,
        "estrellas": estrellas,
        "reviews": reviews,
        'descripcion': descripcion,
        "especificaciones": especs,
        "categoria": categoria,
        "categoria_path": categoria_path,
        'sku': p.get('id'),
        "url": p.get("url"),
        'imagen': imagen
    }

def generate_df(productos, tienda):
    return pd.DataFrame(data=list(transform_data(p, tienda) for p in productos))

def proceso(path, tienda):
    # 1
    data = cargar_json(path)

    # 2
    productos = select_products(data)

    # 3
    return generate_df(productos, tienda)


In [109]:
import os

archivos = list(archivo for archivo in os.listdir('datos/lenovo') if archivo.endswith('.json'))
dataframes = []
tienda = 'Lenovo'

for a in archivos:
    path = os.path.join('datos\lenovo', a)
    dataframes.append(proceso(path, tienda))

In [110]:
for i in range(1, 30):
    try:
        path = 'datos/lenovo/notebooks/notebooks-{}.json'.format(i)
        tienda = 'Lenovo'

        _df = proceso(path, tienda)
        dataframes.append(_df)
        print('listo', path)

    except Exception as e:
        print(e)
        break

listo datos/lenovo/notebooks/notebooks-1.json
listo datos/lenovo/notebooks/notebooks-2.json
listo datos/lenovo/notebooks/notebooks-3.json
listo datos/lenovo/notebooks/notebooks-4.json
listo datos/lenovo/notebooks/notebooks-5.json
listo datos/lenovo/notebooks/notebooks-6.json
listo datos/lenovo/notebooks/notebooks-7.json
listo datos/lenovo/notebooks/notebooks-8.json
listo datos/lenovo/notebooks/notebooks-9.json
listo datos/lenovo/notebooks/notebooks-10.json
listo datos/lenovo/notebooks/notebooks-11.json
listo datos/lenovo/notebooks/notebooks-12.json
listo datos/lenovo/notebooks/notebooks-13.json
listo datos/lenovo/notebooks/notebooks-14.json
listo datos/lenovo/notebooks/notebooks-15.json
listo datos/lenovo/notebooks/notebooks-16.json
listo datos/lenovo/notebooks/notebooks-17.json
listo datos/lenovo/notebooks/notebooks-18.json
listo datos/lenovo/notebooks/notebooks-19.json
listo datos/lenovo/notebooks/notebooks-20.json
'NoneType' object is not iterable


In [113]:
df = pd.concat(dataframes).reset_index(drop=True)
df.shape

C:\Users\uriel\AppData\Local\Temp\ipykernel_26872\2783889678.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dataframes).reset_index(drop=True)


(310, 14)

In [121]:
df.sku.value_counts()

sku
21STCTO1WWAR2_f31ab1f1b6026-4f65-9ed9-0142293b720b    3
83GS00TNAR_f31ab1f1b6026-4f65-9ed9-0142293b720b       3
83HV003AAR_f31ab1f1b6026-4f65-9ed9-0142293b720b       3
21T0CTO1WWAR2_f31ab1f1b6026-4f65-9ed9-0142293b720b    3
83JXCTO1WWAR1_f31ab1f1b6026-4f65-9ed9-0142293b720b    3
                                                     ..
83F2CTO1WWAR1_f31ab1f1b6026-4f65-9ed9-0142293b720b    1
83F5CTO1WWAR1_f31ab1f1b6026-4f65-9ed9-0142293b720b    1
83EYCTO1WWAR1_f31ab1f1b6026-4f65-9ed9-0142293b720b    1
83F5CTO1WWAR2_f31ab1f1b6026-4f65-9ed9-0142293b720b    1
83EYCTO1WWAR2_f31ab1f1b6026-4f65-9ed9-0142293b720b    1
Name: count, Length: 202, dtype: int64

In [127]:
df = df.drop_duplicates(subset='sku').reset_index(drop=True)

In [129]:
df.to_csv('datos/lenovo/productos.csv', index=False)